## Import PMG XML to Dataframe

In [2]:
%pip install lxml
import pandas as pd
# import numpy as np
from lxml import etree


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.7 MB/s eta 0:00:0031m31.4 MB/s eta 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
#import filenames of directory documents/pmg to pandas dataframe

import os

print("Current working directory:", os.getcwd())
path = '~/Dropbox/500_dev/dev_dvv/dvv_content/pmg/'
full_path = os.path.expanduser(path)
files = os.listdir(full_path)

print(f"Inserted {len(files)} files")


Current working directory: /Users/mweber/dev/dvvinsight_dev
Inserted 3478 files


In [10]:
%pip install python-dotenv
%pip install pymongo
import os
from dotenv import load_dotenv

from pymongo import MongoClient
from bson import ObjectId

# Init MongoDB Client
load_dotenv()
mongoClient = MongoClient(os.environ.get('MONGO_URI_DVV'))
database = mongoClient.dvv_content_pool
collection = database.dvv_artikel


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.0/920.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymongo]━━━ 1/2 [pymongo]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
print(collection)

Collection(Database(MongoClient(host=['ac-fzhvzuu-shard-00-01.aev3rng.mongodb.net:27017', 'ac-fzhvzuu-shard-00-02.aev3rng.mongodb.net:27017', 'ac-fzhvzuu-shard-00-00.aev3rng.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-gxqdk3-shard-0', tls=True), 'dvv_content_pool'), 'dvv_artikel')


In [13]:
#---------------------------------------------
# 22.06.2024 added fields quelle_id_embeddings, text_length
# 25.10.2024 added schlagworte
# 18.03.2024 added safe_int
#---------------------------------------------

from datetime import datetime

def safe_int(x: str = "") -> int:
    try:
        return int(x)
    except:
        return 0


data_collection = []

for file in files:

    # nsh - no attribute text
    # raili - no attribute jahrgang
    if file[:3] in ["nsh", "rai"]:
        continue

    if file[-3:] != "xml":
        continue

    if collection.find_one({"dateiname": file}):
        continue

    tree = etree.parse(full_path + f'{file}')
    root = tree.getroot()

    data = []
    for elem in root.findall('artikel'):
        print(f"Processing {file} {elem.find('metadaten/artikel-id').text}")
        
        # Metadaten auslesen
        artikel_id = elem.find('metadaten/artikel-id').text
        quelle_id = elem.find('metadaten/quelle/quelle-id').text
        jahrgang = safe_int(elem.find('metadaten/quelle/jahrgang').text)
        nummer = safe_int(elem.find('metadaten/quelle/nummer').text)
        datum = elem.find('metadaten/quelle/datum').text
        seite_start = safe_int(elem.find('metadaten/quelle/seite-start').text)
        seite_ende = safe_int(elem.find('metadaten/quelle/seite-ende').text)

        # Inhalt auslesen
        title = elem.find('inhalt/titel-liste/titel').text
        if elem.find('inhalt/titel-liste/untertitel') is not None:
            untertitel = elem.find('inhalt/titel-liste/untertitel').text
        else:
            untertitel = "leer"
        text_struktur = elem.find('inhalt/text')
        text_content = "".join(text_struktur.itertext())
        text_length = len(text_content)

        # Felder hinzufügen
        ki_abstract = ""
        date = datetime.strptime(datum, "%d%m%Y")
        text_embeddings = []
        ki_embeddings = []
        quelle_id_embeddings = []
        schlagworte = []
        

        # if quelle_id[:3] == "DVZ":
        data.append([file, artikel_id, quelle_id, jahrgang, nummer, datum, seite_start, seite_ende, title, untertitel, text_content, ki_abstract, date, text_embeddings, ki_embeddings, quelle_id_embeddings, text_length, schlagworte])
    
    data_collection.extend(data)

# read xml file and convert it to a pandas dataframe
df = pd.DataFrame(data_collection, columns=['dateiname', 'artikel_id', 'quelle_id', 'jahrgang', 'nummer', 'datum', 'seite_start', 'seite_ende', 'titel', 'untertitel', 'text', 'ki_abstract', 'date', 'text_embeddings', 'ki_embeddings', 'quelle_id_embeddings', 'text_length', 'schlagworte'])
# df.head(20)
print(f"Inserted {len(df)} records into DataFrame")

# print list of quelle_id grouped by count sorted by quelle_id
print(df.groupby('quelle_id').size().sort_values(ascending=False))


Processing 7592079-RABUS-07042025.xml 7522731
Processing 7592079-RABUS-07042025.xml 7522812
Processing 7592079-RABUS-07042025.xml 7523271
Processing 7592079-RABUS-07042025.xml 7523587
Processing 7592079-RABUS-07042025.xml 7523646
Processing 7592079-RABUS-07042025.xml 7523804
Processing 7592079-RABUS-07042025.xml 7523878
Processing 7592079-RABUS-07042025.xml 7540457
Processing 7592079-RABUS-07042025.xml 7540494
Processing 7592079-RABUS-07042025.xml 7540597
Processing 7592079-RABUS-07042025.xml 7540634
Processing 7592079-RABUS-07042025.xml 7540721
Processing 7592079-RABUS-07042025.xml 7540824
Processing 7592079-RABUS-07042025.xml 7540883
Processing 7592079-RABUS-07042025.xml 7541063
Processing 7592079-RABUS-07042025.xml 7541226
Processing 7592079-RABUS-07042025.xml 7541329
Processing 7592079-RABUS-07042025.xml 7541741
Processing 7592079-RABUS-07042025.xml 7542081
Processing 7592079-RABUS-07042025.xml 7542557
Processing 7592079-RABUS-07042025.xml 7542725
Processing 7592079-RABUS-07042025.

## MongoDB: Import Articles

In [14]:
# Insert data into MongoDB usinf insert_many
data_input = df.to_dict(orient='records')
collection.insert_many(data_input)

print(f"Inserted {len(data_input)} records into MongoDB")

mongoClient.close()

Inserted 1479 records into MongoDB


## Add embeddings & keywords

In [17]:
import os
from dotenv import load_dotenv

from pymongo import MongoClient
from bson import ObjectId

# Init MongoDB Client
load_dotenv()
mongoClient = MongoClient(os.environ.get('MONGO_URI_DVV'))
database = mongoClient.dvv_content_pool
collection = database.dvv_artikel

In [18]:
cursor = collection.dvv_artikel.find({'schlagworte': []}).limit(5)
cursor_list = list(cursor)
print(len(cursor_list))
for record in cursor_list:
    print(f"ID: {record['_id']}")


0


In [25]:
%pip install openai
%pip install -q -U google-generativeai
%pip install groq
%pip install ollama
%pip install torch
import modules.ask_mongo as ask_mongo
# from .autonotebook import tqdm as notebook_tqdm


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached torch-2.8.0-cp312-none-macosx_11_0_arm64.whl.metadata (30 kB)
Using cached torch-2.8.0-cp312-none-macosx_11_0_arm64.whl (73.6 MB)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the ke

In [28]:
ask_mongo.generate_embeddings(input_field="text", output_field="text_embeddings", max_iterations=50000)
# ask_mongo.generate_keywords(input_field="ki_abstract", output_field="schlagworte", max_iterations=5)
# ask_mongo.generate_entities(input_field="ki_abstract", output_field="schlagworte", max_iterations=10)



Generated embeddings for 50000 records.


## Manage Collection